# Merge of Dundee files

#### 1. Read all csv's and store in pandas df

In [1]:
import pandas as pd
import glob
import numpy

In [2]:
path =r'/home/ar/Google/DTU/Courses/Advanced business analytics/ABA - Project /Datasets/DUndee'#path of stored csv files
filenames = glob.glob(path + "/*.csv")
print(filenames)

['/home/ar/Google/DTU/Courses/Advanced business analytics/ABA - Project /Datasets/DUndee/charge-sessions-june-sept.csv', '/home/ar/Google/DTU/Courses/Advanced business analytics/ABA - Project /Datasets/DUndee/cpdata.csv', '/home/ar/Google/DTU/Courses/Advanced business analytics/ABA - Project /Datasets/DUndee/cp-data-dec-mar-2018.csv', '/home/ar/Google/DTU/Courses/Advanced business analytics/ABA - Project /Datasets/DUndee/cp-locations_enriched.csv', '/home/ar/Google/DTU/Courses/Advanced business analytics/ABA - Project /Datasets/DUndee/cp-locations.csv', '/home/ar/Google/DTU/Courses/Advanced business analytics/ABA - Project /Datasets/DUndee/cp-data-mar-may-2018.csv']


Create two dataframes:
1 - with all charge events from Dundee
2 - with the enriched location information from Dundee

In [3]:
df=pd.DataFrame()

for file in filenames:
    if "data" in file or "session" in file:
        data = pd.read_csv(file)
        df = pd.concat([df,data])
        
loc=pd.DataFrame()
for file in filenames:
    if "enriched" in file:
        data=pd.read_csv(file)
        loc = pd.concat([loc, data])

Store a subset of location information on new dataframe to then merge it with the charge event merged dataframe

In [4]:
#separate subset into new dataframe
loc_restricted = loc[['CP ID', 'Latitude', 'Longitude']]
#drop NaN values from loc_restricted and df dataframes
df = df.dropna(subset=['CP ID'])
loc_restricted=loc_restricted.dropna(subset=['CP ID'])

In [6]:

#df['CP ID'] = df['CP ID'].astype('Int64') #THis will give problems, when running all for first time change str by int. Then put str again. PRoblem with pandas converting eveything to floats at the beginnig and then when converting initially to string it causes all Long and latitude values to be NaN
#loc_restricted['CP ID'] = loc_restricted['CP ID'].astype('Int64')

df['CP ID'] = df['CP ID'].astype(str) #THis will give problems, when running all for first time change str by int. Then put str again. PRoblem with pandas converting eveything to floats at the beginnig and then when converting initially to string it causes all Long and latitude values to be NaN
loc_restricted['CP ID'] = loc_restricted['CP ID'].astype(str)


merged_df = pd.merge(df, loc_restricted,  on='CP ID', how='left')#change to inner to keep only rows common on both df's by 'CP ID'

In [7]:
merged_df = merged_df.dropna(subset=['Start Time', 'End Time', 'Start Date', 'End Date'])

# Combine Date and Time into a single datetime column
merged_df['Start Date'] = pd.to_datetime(merged_df['Start Date'] + ' ' + merged_df['Start Time'], format='%d/%m/%Y %H:%M', errors='coerce')
merged_df['End Date'] = pd.to_datetime(merged_df['End Date'] + ' ' + merged_df['End Time'], format='%d/%m/%Y %H:%M', errors='coerce')

merged_df = merged_df.dropna(subset=['Start Time'])
merged_df = merged_df.dropna(subset=['End Time'])

merged_df = merged_df.drop('Start Time', axis=1)
merged_df = merged_df.drop('End Time', axis=1)
merged_df = merged_df.drop('Unnamed: 13', axis=1)
merged_df = merged_df.drop('Unnamed: 14', axis=1)
merged_df = merged_df.drop('Unnamed: 15', axis=1)


In [8]:
#Create a new column with total duration of charging event by substracting end date and start date
merged_df['Total Duration (hh:mm:ss)'] = merged_df['End Date'] - merged_df['Start Date']

In [9]:
merged_df.head()
merged_df.columns

Index(['Charging event', 'User ID', 'CP ID', 'Connector', 'Start Date',
       'End Date', 'Total kWh', 'Cost', 'Site', 'Group', 'Model', 'Latitude',
       'Longitude', 'Total Duration (hh:mm:ss)'],
      dtype='object')

In [10]:
column_mapping={
    'Charging event': 'Transaction ID',
    'User ID': 'User ID',
    'CP ID': 'EVSE ID',
    'Connector': 'Plug Type',
    #'Start Date': 'Start Date',
    #'End Date': 'End date', 
    'Total kWh': 'Energy(kWh)', 
    'Cost': 'drop', 
    'Site':'Address 1', 
    'Group': 'drop',
    'Model': 'Port Type', 
    #'Latitude':'Latitude',
    #'Longitude':'Longitude'
}

In [11]:
#Drop columns with value key drop
columns_to_drop = [key for key, value in column_mapping.items() if value == 'drop']
merged_df = merged_df.drop(columns=columns_to_drop, errors='ignore')
# Replace name of columns
merged_df = merged_df.rename(columns={k: v for k, v in column_mapping.items() if v != 'drop'})
merged_df = merged_df.reindex(merged_df.columns, axis=1)

In [12]:
merged_df.head()
merged_df.columns

Index(['Transaction ID', 'User ID', 'EVSE ID', 'Plug Type', 'Start Date',
       'End Date', 'Energy(kWh)', 'Address 1', 'Port Type', 'Latitude',
       'Longitude', 'Total Duration (hh:mm:ss)'],
      dtype='object')

In [13]:
merged_df.to_csv('Dundee_merged', index=False)